# Projeto Inteligência Artificial e Previsões

Neste projeto, iremos otimizar uma inteligência artificial para identificar scores de crédito dos clientes de um banco, tendo em vista a eficiência das classificações para minimizar a perda do banco.

In [9]:
import pandas as pd

table = pd.read_csv("clientes.csv")

display(table)

,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,3392,1,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,265.0,21.465380,alto_gasto_pagamento_baixos,312.494089,Good,1,1,1,1,0
1,3392,2,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,266.0,21.465380,baixo_gasto_pagamento_alto,284.629162,Good,1,1,1,1,0
2,3392,3,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,267.0,21.465380,baixo_gasto_pagamento_medio,331.209863,Good,1,1,1,1,0
3,3392,4,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,5.0,...,268.0,21.465380,baixo_gasto_pagamento_baixo,223.451310,Good,1,1,1,1,0
4,3392,5,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,6.0,...,269.0,21.465380,alto_gasto_pagamento_medio,341.489231,Good,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,37932,4,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,23.0,...,378.0,24.028477,alto_gasto_pagamento_alto,479.866228,Poor,1,0,0,0,1
99996,37932,5,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,18.0,...,379.0,24.028477,alto_gasto_pagamento_medio,496.651610,Poor,1,0,0,0,1
99997,37932,6,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,27.0,...,380.0,24.028477,alto_gasto_pagamento_alto,516.809083,Poor,1,0,0,0,1
99998,37932,7,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,20.0,...,381.0,24.028477,baixo_gasto_pagamento_alto,319.164979,Standard,1,0,0,0,1


In [10]:
display(table.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  object 
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  object 
 13  divida_total              100000 non-null  fl

None

# Tratamento dos Dados


In [11]:
table = table.drop("id_cliente", axis = 1)

In [13]:
display(table.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   mes                       100000 non-null  int64  
 1   idade                     100000 non-null  float64
 2   profissao                 100000 non-null  object 
 3   salario_anual             100000 non-null  float64
 4   num_contas                100000 non-null  float64
 5   num_cartoes               100000 non-null  float64
 6   juros_emprestimo          100000 non-null  float64
 7   num_emprestimos           100000 non-null  float64
 8   dias_atraso               100000 non-null  float64
 9   num_pagamentos_atrasados  100000 non-null  float64
 10  num_verificacoes_credito  100000 non-null  float64
 11  mix_credito               100000 non-null  object 
 12  divida_total              100000 non-null  float64
 13  taxa_uso_credito          100000 non-null  fl

None

Observamos algumas colunas com valores no formato "object", ou seja, possuem textos, poré, o modelo de classificação não consegue ler esse formato, então devemos transformar em formatos numéricos:

In [16]:
from sklearn.preprocessing import LabelEncoder
#transforma a coluna de texto em números

codificador = LabelEncoder()

for coluna in table.columns:
    if table[coluna].dtype == "object" and coluna != "score_credito":
        table[coluna] = codificador.fit_transform(table[coluna])

#foi transformado todas as colunas de texto, menos o "score_credito", por ser o resultado da operação e não um algo que vamos utilizar


# Modelo de Classificação

In [18]:
#agora escolhemos as colunas para realizar a previsão, onde x são as colunas que auxiliarão para o processo e y o resultado esperado

x = table.drop("score_credito", axis=1)
y = table["score_credito"]

from sklearn.model_selection import train_test_split

#agora separamos os dados em treino e teste. Treino serão os dados para o modelo aprender e teste para o modelo verificar o aprendizado.

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state=1)


Utilizaremos 2 modelos para classificar os dados, o primeiro será o Modelo de Árvores de Decisão e o segundo o Modelo do Vizinho Mais Próximo, conhecido como KNN:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

modelo_arvore = RandomForestClassifier() #modelo árvore de decisão
modelo_knn = KNeighborsClassifier() # modelo KNN

modelo_arvore.fit(x_treino, y_treino)
modelo_knn.fit(x_treino, y_treino)


# Teste de Eficácia

Agora, podemos utilizar a base de dados de teste para medir a eficácia do modelo, utilizando o método de acurácia, verificando quantas informações foram previstas de forma correta. Um exemplo dessa medição está descrita abaixo:

In [21]:
#aqui vamos testar a acurácia de um hipotético modelo que chuta todos os valores como "Standard"

contagem_scores = table["score_credito"].value_counts()
print(contagem_scores["Standard"] / sum(contagem_scores))

0.53174


Então vemos que a acurácia desse modelo hipotético foi de 0.53%, vamos realizar a mesma operação, utilizando uma biblioteca, para os modelos reais.

In [24]:
from sklearn.metrics import accuracy_score

#Processo de testagem
previsao_arvore = modelo_arvore.predict(x_teste)
previsao_knn = modelo_knn.predict(x_teste.to_numpy())

#Realizando a acurácia do teste
print(round(accuracy_score(y_teste, previsao_arvore), 3))
print(round(accuracy_score(y_teste, previsao_knn), 3))



/home/edmurcn/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


0.824
0.732


Desse modo, vemos que o modelo de árvore de decisão foi mais eficiente neste caso, realizando uma previsão de 82% dos scores, contra 73% do método de KNN. Com isso, o banco maximizará suas decisões corretas. Além de poupar tempo realizado esta tarefa manualmente.


Podemos testar o modelo com dados diferentes e obter os resultados das classificações dos clientes:

In [28]:
novos_clientes = pd.read_csv("novos_clientes.csv")
display(novos_clientes)

#tratando os dados
for coluna in novos_clientes.columns:
    if novos_clientes[coluna].dtype == "object":
        novos_clientes[coluna] = codificador.fit_transform(novos_clientes[coluna])

#os dados novos já estão tratados, então aplicamos o método para classificação
classificacao_nova = modelo_arvore.predict(novos_clientes)
print(classificacao_nova)


,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,num_pagamentos_atrasados,...,taxa_uso_credito,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,1,31.0,empresario,19300.340,6.0,7.0,17.0,5.0,52.0,19.0,...,29.934186,218.0,44.50951,baixo_gasto_pagamento_baixo,312.487689,1,1,0,0,0
1,4,32.0,advogado,12600.445,5.0,5.0,10.0,3.0,25.0,18.0,...,28.819407,12.0,0.00000,baixo_gasto_pagamento_medio,300.994163,0,0,0,0,1
2,2,48.0,empresario,20787.690,8.0,6.0,14.0,7.0,24.0,14.0,...,34.235853,215.0,0.00000,baixo_gasto_pagamento_alto,345.081577,0,1,0,1,0


['Poor' 'Standard' 'Standard']


Então vemos que para um novo banco de clientes, obtemos classificações satisfatórias.

# Informações Adicionais

Por fim, podemos também obter informações a respeito das características mais importantes para a classificação do score de crédito:

In [30]:
colunas = list(x_teste.columns)
importancia = pd.DataFrame(index=colunas, data = modelo_arvore.feature_importances_)
importancia = importancia * 100
print(importancia)

                                  0
mes                        3.921995
idade                      4.269011
profissao                  3.312302
salario_anual              5.120212
num_contas                 3.428259
num_cartoes                4.364131
juros_emprestimo           8.286615
num_emprestimos            2.755776
dias_atraso                6.612524
num_pagamentos_atrasados   4.679934
num_verificacoes_credito   4.363217
mix_credito                8.197059
divida_total              11.754348
taxa_uso_credito           5.080308
idade_historico_credito    7.697595
investimento_mensal        4.817366
comportamento_pagamento    2.366489
saldo_final_mes            5.427170
emprestimo_carro           0.703259
emprestimo_casa            0.737691
emprestimo_pessoal         0.693707
emprestimo_credito         0.708796
emprestimo_estudantil      0.702236


Vemos que as características que mais contribuem para a classificação do score são, os juros empréstimos, o mix crédito e a divida total.

# Conclusão

Neste projeto realizamos o processo de aprendizado de máquina para classificar o score de crédito de clientes de uma empresa por meio das informações fornecidas no cadastro do indivíduo. Treinamos uma Inteligência Artificial por meio de dois métodos (Método Árvore de Decisão) e (Método KNN). Medimos a acurácia dos métodos e obtivemos um resultado mais eficiente com o método de árvore de decisão, podendo prever 82% dos scores corretamente. Além disso, após treinada, a inteligência nos concedeu as características mais relevantes para a classificação dos scores, com base nos dados fornecidos para o treinamento.